In [7]:
import numpy as np
import pandas as pd
from utils import *
from lightgbm import LGBMClassifier
from feature_engineering import *
from check_score.scorer import Kaggle_score, get_model_score

# LGBM baseline

In [8]:
df_train = get_data_train()
df_test = get_data_test()

X_train = df_train.drop(columns=["Cover_Type"], axis=1)
y_train = df_train["Cover_Type"]

In [3]:
c_w ={1: 0.4, 2: 0.45, 3: 0.04,4: 0.01, 5: 0.04, 6: 0.04, 7: 0.04},

lgbm = LGBMClassifier(
    objective='multiclass',
    num_class=7,
    class_weight=c_w,
    boosting_type='gbdt',
    num_leaves=31,
    n_estimators=300,
    verbose=-1,
    n_jobs=-1
)

print(IWCV(df_train, lgbm, k_valid=10))

get_model_score(lgbm, df_train)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

## Soil Type

In [ ]:
def preprocess(df):
    df = concat_Soil_Type(df, new_name='ST_concat', drop_value=True) 
    df = concat_Wilderness_Area(df, new_name='WA_concat', drop_value=True) 
    df = group_climatic_zone(df)
    df = group_geological_zone(df)
    return df

df_train_new = preprocess(df_train)
cat_col = [ 'ST_concat', 'WA_concat','climatic_zone', 'geological_zone']

X_train = df_train_new.drop(columns=["Cover_Type"], axis=1)
y_train = df_train_new["Cover_Type"]
lgbm.fit(X_train, y_train, categorical_feature=cat_col)
pred = lgbm.predict(df_test)

pred = clean_predictor(y_pred=pred.Cover_Type, Id=pred.Id)
csv_for_submission(pred,"pred_test")
Kaggle_score('Output/pred_test')

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,ST_concat,geological_zone
0,242642,2881,130,22,210,54,1020,250,221,88,...,0,0,0,0,0,0,0,1,30,7
1,309891,3005,351,14,242,-16,1371,194,215,159,...,0,0,0,0,0,0,0,1,24,7
2,287847,3226,63,14,618,2,1092,232,210,107,...,0,0,0,0,0,0,0,1,29,7
3,516307,3298,317,8,661,60,752,198,233,174,...,0,0,0,0,0,0,0,1,23,2
4,124860,3080,35,6,175,26,3705,219,227,144,...,0,0,0,0,0,0,0,1,24,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,475155,3328,321,13,323,12,5109,186,227,180,...,0,0,0,0,1,0,0,7,38,7
15116,514378,3455,37,5,841,92,939,220,229,146,...,0,0,0,0,0,0,1,7,40,0
15117,368425,3279,90,14,404,113,1513,240,218,105,...,0,0,0,0,0,0,0,7,29,7
15118,537844,3589,357,9,418,52,1868,205,223,155,...,0,0,0,0,0,0,1,7,40,0


## New features

In [1]:
ST = {
    1: {
        "ELU": 2702,
        "family":"Cathedral", 
        "complex": "Rock outcrop complex", 
        "stone":"extremely stony",
        },
    2: {
        "ELU": 2703,
        "family":"Vanet",
        "complex": "Ratake families complex", 
        "stone":"very stony",
        },
    3: {
        "ELU": 2704,
        "family":"",#Haploborolis - 
        "complex":"Rock outcrop complex",
        "stone":"rubbly",
        },
    4: {
        "ELU": 2705,
        "family":"Ratake", 
        "complex": "Rock outcrop complex", 
        "stone":"rubbly",
        },
    5: {
        "ELU": 2706,
        "family":"Vanet",
        "complex":"Rock outcrop complex complex",
        "stone":"rubbly",
        },
    6: {
        "ELU": 2717,
        "family":"",#Vanet - Wetmore families - 
        "complex":"Rock outcrop complex", 
        "stone":"stony",
        },
    7: {
        "ELU": 3501,
        "family":"Gothic",
        "complex":"",
        "stone":"",
        },
    8: {
        "ELU": 3502,
        "family":"",#Supervisor - 
        "complex":"Limber families complex",
        "stone":"",
        },
    9: {
        "ELU": 4201,
        "family":"Troutville", 
        "stone":"very stony",
        },
    10: {
        "ELU": 4703,
        "family":"Catamount", #Bullwark - 
        "complex":"Rock outcrop complex", 
        "stone":"rubbly",
        },
    11: {
        "ELU": 4704,
        "family":"Catamount",# Bullwark - 
        "complex":"Rock land complex",
        "stone":"rubbly",
        },
    12: {
        "ELU": 4744,
        "family":"Legault",
        "complex":"Rock land complex",
        "stone":"stony",
        },
    13: {
        "ELU": 4758,
        "family":"Catamount",# - Rock land - Bullwark family complex,
        "stone":"rubbly",
        },
    14: {
        "ELU": 5101,
        "family":"",#Pachic Argiborolis - 
        "complex":"Aquolis complex",
        "stone":"",
        },
    15: {
        "ELU": 5151,
        "family":None,
        "stone":None,
        },
    16: {
        "ELU": 6101,
        "family":"",#Cryaquolis - Cryoborolis complex.
        "stone":"",
        },
    17: {
        "ELU": 6102,
        "family":"Gateview", #- Cryaquolis complex.,
        "stone":"",
        },
    18: {
        "ELU": 6731,
        "family":"Rogert",
        "stone":"very stony",
        },
    19: {
        "ELU": 7101,
        "family":"",#Typic Cryaquolis - Borohemists complex.,
        "stone":"",
        },
    20: {
        "ELU": 7102,
        "family":"",#Typic Cryaquepts - Typic Cryaquolls complex.,
        "stone":"",
        },
    21: {
        "ELU": 7103,
        "family":"Leighcan", #Typic Cryaquolls -, till substratum complex.,
        "stone":"",
        },
    22: {
        "ELU": 7201,
        "family":"Leighcan",  #till substratum, 
        "stone":"extremely bouldery",
        },
    23: {
        "ELU": 7202,
        "family":"Leighcan",  #till substratum - Typic Cryaquolls complex.,
        "stone":"",
        },
    24: {
        "ELU": 7700,
        "family":"Leighcan", 
        "stone":"extremely stony",
        },
    25: {
        "ELU": 7701,
        "family":"Leighcan", #warm, 
        "stone":"extremely stony",
        },
    26: {
        "ELU": 7702,
        "family":"Catamount", #Granile -  complex,
        "stone":"very stony",
        },
    27: {
        "ELU": 7709,
        "family":"Leighcan", #warm - 
         "complex":"Rock outcrop complex",
        "stone":"extremely stony",
        },
    28: {
        "ELU": 7710,
        "family":"Leighcan",
        "complex":"Rock outcrop complex", 
        "stone":"extremely stony",
        },
    29: {
        "ELU": 7745,
        "family":"Como",#- Legault families complex, 
        "stone":"extremely stony",
        },
    30: {
        "ELU": 7746,
        "family":"Como", #- Rock land - Legault family complex, 
        "stone":"extremely stony",
        },
    31: {
        "ELU": 7755,
        "family":"Catamount",# Leighcan -  complex, 
        "stone":"extremely stony",
        },
    32: {
        "ELU": 7756,
        "family":"Catamount", # - Rock outcrop - Leighcan family complex,
        "stone":"extremely stony",
        },
    33: {
        "ELU": 7757,
        "family":"Catamount", #Leighcan
        "complex":"Rock outcrop complex",
        "stone":"extremely stony",
        },
    34: {
        "ELU": 7790,
        "family":"",#Cryorthents - 
        "complex":"Rock land complex",
        "stone":"extremely stony",
        },
    35: {
        "ELU": 8703,
        "family":"",#Cryumbrepts - Rock outcrop - Cryaquepts complex.,
        "stone":"",
        },
    36: {
        "ELU": 8707,
        "family":"Bross", #- Rock land - Cryumbrepts complex,
        "stone":"extremely stony",
        },
    37: {
        "ELU": 8708,
        "family":"",#Rock outcrop - Cryumbrepts - Cryorthents complex,
        "stone":"extremely stony",
        },
    38: {
        "ELU": 8771,
        "family":"Moran", #- Leighcan - Cryaquolls complex,
        "stone":"extremely stony",
        },
    39: {
        "ELU": 8772,
        "family":"Moran", # - Cryorthents - Leighcan family complex,
        "stone":"extremely stony",
        },
    40: {
        "ELU": 8776,
        "family":"Moran", #- Cryorthents - 
        "complex":"Rock land complex",
        "stone":"extremely stony",
        },
}

In [9]:
def preprocess(df):
    df = concat_Soil_Type(df, new_name='ST_concat', drop_value=True) 
    df = concat_Wilderness_Area(df, new_name='WA_concat', drop_value=True) 
    df = group_climatic_zone(df)
    df = group_geological_zone(df)
    ELU_new = {k:v['ELU']%100 for k, v in ST.items()}
    df['ELU'] = df['ST_concat'].map(ELU_new)
    family_new = {k:v['family'] for k, v in ST.items()}
    df['family'] = df['ST_concat'].map(family_new)
    stone_new = {k:v['stone'] for k, v in ST.items()}
    df['stone'] = df['ST_concat'].map(stone_new)
    return df

df_train1 = preprocess(df_train)
enc = OneHotEncoder(categories=['ELU', "family", "stone"])
df_train1 = enc.fit_transform(df_train1)
df_test1 = preprocess(df_test)
df_test1 = enc.transform(df_test1)

In [2]:
import pandas as pd
predict_true = pd.read_parquet("ground_truth.parquet")["Cover_Type"]
predict_true.value_counts()
# df_train = pd.read_csv("train.csv")
# df_train.Cover_Type.value_counts()


2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [5]:
a = [283301,211840,35754,20510,17367,9493,2747]
b = [i/sum(a) for i in a]
# print(a[0]/a[1])
# print(80000/60000)
b

[0.48759922342395684,
 0.36460520608868663,
 0.06153745533655071,
 0.03530047572167184,
 0.02989094889606411,
 0.016338733107061472,
 0.004727957426008413]

In [22]:
from sklearn.preprocessing import OneHotEncoder

ELU_new = {k:v['ELU']%100 for k, v in ST.items()}
df_train1['ELU'] = df_train1['ST_concat'].map(ELU_new)
family_new = {k:v['family'] for k, v in ST.items()}
df_train1['family'] = df_train1['ST_concat'].map(family_new)
stone_new = {k:v['stone'] for k, v in ST.items()}
df_train1['stone'] = df_train1['ST_concat'].map(stone_new)

df_train1
enc = OneHotEncoder(categories=['ELU', "family", "stone"])
df_train1 = enc.fit_transform(df_train1)

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).